In [ ]:
import stacrs
from pystac import Item
from odc.stac import load

from odc.geo import Geometry
from odc.geo.xr import mask
from dask.distributed import Client

import geopandas as gpd

In [ ]:
region = ("Belize", "BLZ")

url = (
    "https://media.githubusercontent.com/media/wmgeolab/geoBoundaries/"
    f"9469f09592ced973a3448cf66b6100b741b64c0d/releaseData/gbOpen/BLZ/ADM0/geoBoundaries-{region[1]}-ADM0-all.zip"
)

geometry = gpd.read_file(url, layer="geoBoundaries-BLZ-ADM0")
geometry.explore()

In [ ]:
url = "https://data.ldn.auspatious.com/geo_ls_lp/geo_ls_lp_0_1_0.parquet"

dict_list = stacrs.search(
    url,
    bbox=list(geometry.geometry.bounds.values.flatten())
)

items = [Item.from_dict(d) for d in dict_list]

data = load(items, geopolygon=geometry, chunks={})
data

In [ ]:
masked = mask(data, Geometry(geometry.to_geo_dict()))

In [ ]:
with Client(n_workers=4, threads_per_worker=8):
    loaded = masked.evi2.compute()

In [ ]:
from dea_tools.plotting import xr_animation

loaded_ds = loaded.to_dataset(name="evi2")

animation = xr_animation(
    loaded_ds,
    bands=["evi2"],
    width_pixels=800,
    output_path="belize.mp4",
    interval=300,
    show_date="%Y",
    imshow_kwargs={"cmap": "viridis", "vmin": 0, "vmax": 300},
    show_colorbar=False
)
del animation